In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import os


In [4]:
data = os.path.join(os.getcwd(),'data_final.csv')

In [18]:
df = pd.read_csv(data)
df = df.loc[df['Price movement'].isin(["Up", "Down"])]
df["Headline"].apply(lambda x: x.lower())
df["Headline"].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['Headline'].values)
X = tokenizer.texts_to_sequences(df['Headline'].values)
X = pad_sequences(X)

In [19]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 30, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
Y = pd.get_dummies(df['Price movement']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(411, 30) (411, 2)
(203, 30) (203, 2)


In [27]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
 - 0s - loss: 0.1169 - accuracy: 0.9489
Epoch 2/20
 - 0s - loss: 0.0691 - accuracy: 0.9684
Epoch 3/20
 - 0s - loss: 0.0610 - accuracy: 0.9611
Epoch 4/20
 - 0s - loss: 0.0573 - accuracy: 0.9635
Epoch 5/20
 - 0s - loss: 0.0483 - accuracy: 0.9805
Epoch 6/20
 - 0s - loss: 0.0476 - accuracy: 0.9757
Epoch 7/20
 - 0s - loss: 0.0538 - accuracy: 0.9659
Epoch 8/20
 - 0s - loss: 0.0467 - accuracy: 0.9708
Epoch 9/20
 - 0s - loss: 0.0435 - accuracy: 0.9732
Epoch 10/20
 - 0s - loss: 0.0471 - accuracy: 0.9757
Epoch 11/20
 - 0s - loss: 0.0599 - accuracy: 0.9586
Epoch 12/20
 - 0s - loss: 0.0521 - accuracy: 0.9659
Epoch 13/20
 - 0s - loss: 0.0466 - accuracy: 0.9732
Epoch 14/20
 - 0s - loss: 0.0463 - accuracy: 0.9684
Epoch 15/20
 - 0s - loss: 0.0445 - accuracy: 0.9757
Epoch 16/20
 - 0s - loss: 0.0467 - accuracy: 0.9684
Epoch 17/20
 - 0s - loss: 0.0473 - accuracy: 0.9659
Epoch 18/20
 - 0s - loss: 0.0523 - accuracy: 0.9659
Epoch 19/20
 - 0s - loss: 0.0461 - accuracy: 0.9781
Epoch 20/20
 - 0s - l

In [28]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.56
acc: 0.55
